In [ ]:
#!pip3 install --upgrade transformers
!pip install simpletransformers

In [56]:
!pip install textfeatures

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#!pip3 install sentencepiece

In [1]:
import pandas as pd
import numpy as np
import sklearn
from tqdm import tqdm, notebook
from tqdm import tqdm
import textfeatures as tf

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sofia.zabit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:

df = pd.read_csv('twitter_sexism_parsed_dataset.csv',encoding = 'iso-8859-1')#,names=['id', 'text', 'ICD_Chapter', 'label'])
df.head()

,index,id,Text,Annotation,oh_label
0,5.35198627292254E+017,5.35198627292254E+017,RT @BeepsS: @senna1 @BeepsS: I'm not sexist bu...,sexism,1.0
1,5.75984924030714E+017,5.75984924030714E+017,There's some very hate able teams this year #MKR,none,0.0
2,5.7233536016588E+017,5.7233536016588E+017,"RT @The_Eccles: ""Everyone underestimated us"" \...",none,0.0
3,5.72337925708374E+017,5.72337925708374E+017,RT @NOTLukeDarcy: did @Channel7 or #MKR actual...,none,0.0
4,4.43033024528011E+017,4.43033024528011E+017,"No, you don't. @Shut_Up_Jeff: I thought of a r...",sexism,1.0


In [3]:
df.columns

Index(['index', 'id', 'Text', 'Annotation', 'oh_label'], dtype='object')

In [4]:
df = df.iloc[:,2:]

In [5]:
df['Text']=df['Text'].fillna("")


df['oh_label'] = df['oh_label'].fillna(0)

In [6]:
df = df[['Text', 'oh_label']]

In [7]:
df.rename(columns={'oh_label':'label'}, inplace = True)

In [8]:
# Removed the stopword
tf.stopwords(df,"Text","stopword")
df[["Text","stopword"]].head()

,Text,stopword
0,RT @BeepsS: @senna1 @BeepsS: I'm not sexist bu...,"[not, but, if, you're, a, and, you, your]"
1,There's some very hate able teams this year #MKR,"[some, very, this]"
2,"RT @The_Eccles: ""Everyone underestimated us"" \...","[as, as, the]"
3,RT @NOTLukeDarcy: did @Channel7 or #MKR actual...,"[did, or, if, any, of, these]"
4,"No, you don't. @Shut_Up_Jeff: I thought of a r...","[you, of, a, and, not, but, have, to]"


In [9]:
tf.clean(df,"Text","CText")
df[["Text","CText"]].head()

,Text,CText
0,RT @BeepsS: @senna1 @BeepsS: I'm not sexist bu...,beepss senna beepss sexist fuck youre woman ca...
1,There's some very hate able teams this year #MKR,theres hate able teams year
2,"RT @The_Eccles: ""Everyone underestimated us"" \...",the_eccles everyone underestimated still well ...
3,RT @NOTLukeDarcy: did @Channel7 or #MKR actual...,notlukedarcy channel actually check people cou...
4,"No, you don't. @Shut_Up_Jeff: I thought of a r...",dont shut_up_jeff thought really funny joke pr...


In [10]:
df.CText = str(df['CText']).rstrip()

In [11]:
df.label = pd.to_numeric(df.label, errors='coerce')
print(df.dtypes)

Text         object
label       float64
stopword     object
CText        object
dtype: object


In [12]:
sorted(df.label.unique())
num_labels = df.label.nunique()
num_labels

2

In [13]:
df['label'] = df['label'].fillna(0)

In [14]:
df = df[['CText', 'label']]
df.rename(columns={'CText':'Text'}, inplace = True)

In [15]:
possible_labels = df.label.unique()
print(possible_labels)

[1. 0.]


In [16]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [17]:
df['label'] = df.label.replace(label_dict)

In [18]:
df.label = df.label.astype('int64')
df.Text = str(df['Text']).rstrip()

In [19]:
from sklearn.model_selection import train_test_split
#target = 'label'

train_df, eval_df = train_test_split(df, test_size=0.2)#,stratify=df.label.values)

In [ ]:
from simpletransformers.classification import ClassificationModel

model_args = {
    "num_train_epochs": 1,
    "learning_rate": 3e-5,
    "train_batch_size": 32,
    'overwrite_output_dir':True
}
# Create a ClassificationModel
model = ClassificationModel('roberta', 'distilroberta-base', num_labels=num_labels, use_cuda=False, args=model_args)

C:\Users\sofia.zabit\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [21]:
%%time
# Train the model
model.train_model(train_df)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

C:\Users\sofia.zabit\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Wall time: 3h 18min 50s


(372, 0.5454711423766229)

In [22]:
%%time
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

C:\Users\sofia.zabit\Anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:900: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/2977 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [24]:
from sklearn.metrics import f1_score, accuracy_score, classification_report


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)


  0%|          | 0/2977 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/373 [00:00<?, ?it/s]

C:\Users\sofia.zabit\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [25]:
result

{'mcc': 0.0,
 'tp': 2330,
 'tn': 0,
 'fp': 647,
 'fn': 0,
 'f1': 0.7826671145448438,
 'acc': 0.7826671145448438,
 'eval_loss': 0.5228723155349573}

In [ ]:
model_outputs

In [ ]:
_, predictions, true_vals = model.eval_model(eval_df)

In [ ]:
from pandas import DataFrame
your_list = [predictions, true_vals]
df = DataFrame (your_list)


In [ ]:
df.head()

In [ ]:
df.to_csv('check.csv')

In [ ]:
predictions = []
for x in model_outputs:
    predictions.append(np.argmax(x))

#print('f1 score:', f1_score(eval_df['label'], predictions,average='micro'))

#target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(eval_df['label'], predictions,sorted(possible_labels)))#,target_names=sorted(df['label'].unique())))


In [ ]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(eval_df['label'], predictions,sorted(possible_labels))

In [ ]:
import seaborn as sns
sns.heatmap(cf, annot=True)

In [ ]:
sns.set(font_scale=1.4) # for label size
sns.heatmap(cf, annot=True, annot_kws={"size": 14}, cmap='Blues') # font size


In [ ]:
eval_df.to_csv('eval.csv')

In [ ]:
pred = pd.DataFrame(predictions, eval_df)
pred.to_csv('pred_chk.csv')

In [ ]:
#from sklearn import metrics
#print(metrics.classification_report(eval_df['label'], predictions, target_names=df['label'].unique()))

In [ ]:
#model.train_model(train_df)

#result, model_outputs, wrong_predictions = model.eval_model(eval_df)

#lst = []
#for arr in model_outputs:
#    lst.append(np.argmax(arr))
#true = eval_df['label'].tolist()
#predicted = lst
#sklearn.metrics.accuracy_score(true,predicted)

#https://stackoverflow.com/questions/64595546/higher-accuracy-using-simpletransformers-vs-transformers-library-with-bert

In [ ]:
#print(metrics.classification_report(true_vals, predictions, target_names=df['label'].unique()))
#https://github.com/shudima/notebooks/blob/master/BERT_to_the_rescue.ipynb


#model.train_model(train_df)

#result, model_outputs, wrong_predictions = model.eval_model(eval_df)

#lst = []
#for arr in model_outputs:
#    lst.append(np.argmax(arr))
#true = eval_df['label'].tolist()
#predicted = lst
#sklearn.metrics.accuracy_score(true,predicted)